In [1]:
import torch
import numpy as np
import pandas as pd
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1024, 15, 64, 1
from sklearn.utils import shuffle

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [6]:
df = pd.read_csv('train.csv') ## Import data

reqd_feat = ['LotFrontage','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtUnfSF','1stFlrSF','2ndFlrSF','GrLivArea','Fireplaces','GarageCars','YrSold']
df = df[['LotFrontage','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtUnfSF','1stFlrSF','2ndFlrSF','GrLivArea','Fireplaces','GarageCars','YrSold','SalePrice']]
a = df.to_numpy()
a[np.isnan(a).any(axis=1)] = 0
scalingF = a.max(axis=0)[-1]
print(scalingF)
a = a/a.max(axis=0)
col = [i for i in df] ## IDs of the house. Irrelevant
print(col)
data = a[:,0:-1]
# data = data/data.max(axis=0)
labe = a[:,-1]
print(len(data),len(data[0]))
print(data)
print(labe)
## Data preprocessing

755000.0
['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'Fireplaces', 'GarageCars', 'YrSold', 'SalePrice']
1460 15
[[0.20766773 0.03925759 0.7        ... 0.         0.5        0.99900498]
 [0.25559105 0.04460034 0.6        ... 0.33333333 0.5        0.99850746]
 [0.2172524  0.05226602 0.7        ... 0.33333333 0.5        0.99900498]
 ...
 [0.21086262 0.04200794 0.7        ... 0.66666667 0.25       1.        ]
 [0.2172524  0.04514391 0.5        ... 0.         0.25       1.        ]
 [0.23961661 0.046166   0.5        ... 0.         0.25       0.99900498]]
[0.27615894 0.24039735 0.29602649 ... 0.35298013 0.18824503 0.19536424]


In [3]:
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(3000):
    j = 0
    data, labe = shuffle(data, labe)
    while j < len(a):
        x = data[j:j+N]
        x = torch.tensor(x, dtype=torch.float32)
        y = labe[j:j+N]
        y = torch.tensor(y, dtype=torch.float32)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        j += N
    print(t, loss.item())

0 8064.185546875
1 4494.8583984375
2 5928.7451171875
3 5828.21142578125
4 4232.3232421875
5 3365.88720703125
6 3283.465087890625
7 4949.97412109375
8 4101.669921875
9 3900.918212890625
10 3815.722412109375
11 3732.075439453125
12 3508.9111328125
13 3676.65625
14 3289.919921875
15 3365.8896484375
16 3021.67431640625
17 3187.079833984375
18 3478.189697265625
19 3223.49755859375
20 3618.726318359375
21 3650.815673828125
22 3364.30322265625
23 3975.901611328125
24 3014.29443359375
25 3606.5771484375
26 3529.142578125
27 3290.697998046875
28 3767.07568359375
29 3379.537353515625
30 3068.849365234375
31 3213.283203125
32 4380.0361328125
33 3267.5341796875
34 3489.012451171875
35 3273.826416015625
36 3680.792724609375
37 3432.876953125
38 3279.907958984375
39 4345.65966796875
40 3577.944580078125
41 3611.80419921875
42 3838.143798828125
43 3075.955810546875
44 3554.919921875
45 4324.20947265625
46 2967.9970703125
47 3584.90869140625
48 3635.510498046875
49 3192.373779296875
50 3531.11328125
5

400 2943.728515625
401 3488.306640625
402 3108.242919921875
403 3496.5205078125
404 3408.9140625
405 3865.2802734375
406 3694.836181640625
407 3462.05810546875
408 3997.638671875
409 4093.5234375
410 3439.9599609375
411 3120.92529296875
412 3407.282958984375
413 3266.249755859375
414 3150.5986328125
415 3364.386474609375
416 3281.634033203125
417 3182.493896484375
418 3806.99951171875
419 3581.162109375
420 3662.846923828125
421 3186.258056640625
422 3663.14306640625
423 3322.740478515625
424 3935.08447265625
425 3357.152587890625
426 3406.127685546875
427 3468.744873046875
428 3684.75048828125
429 3294.37158203125
430 3241.3447265625
431 3110.81396484375
432 3441.42236328125
433 3469.420654296875
434 3438.56591796875
435 3587.021240234375
436 4185.76416015625
437 3701.843017578125
438 3127.071044921875
439 3773.404296875
440 3215.924560546875
441 3467.828125
442 3532.133544921875
443 3982.138671875
444 3368.52783203125
445 3703.68798828125
446 3605.882080078125
447 3577.839111328125
4

800 3299.136474609375
801 3574.96484375
802 3094.64111328125
803 3227.592041015625
804 3631.341552734375
805 3315.543212890625
806 3729.2265625
807 3633.283447265625
808 3612.35546875
809 2843.030517578125
810 3544.213134765625
811 3788.6669921875
812 3212.69580078125
813 2983.823486328125
814 3015.170166015625
815 3151.97216796875
816 3531.971923828125
817 3312.9658203125
818 3378.01953125
819 2730.42822265625
820 3458.39892578125
821 3951.348876953125
822 3623.89599609375
823 3546.303466796875
824 3271.238037109375
825 3788.7998046875
826 3610.11572265625
827 3400.10205078125
828 3692.904541015625
829 3694.608154296875
830 3179.034423828125
831 3515.509765625
832 3550.841796875
833 4178.24267578125
834 3837.816162109375
835 2806.86376953125
836 3829.379150390625
837 3305.265869140625
838 3938.828857421875
839 3776.98681640625
840 2877.427490234375
841 3106.6611328125
842 3817.315185546875
843 3233.435546875
844 3443.314697265625
845 3308.391845703125
846 3853.379638671875
847 3305.97

1183 3981.376708984375
1184 3734.797607421875
1185 3204.323974609375
1186 3111.174560546875
1187 3505.173095703125
1188 4210.33154296875
1189 3634.77392578125
1190 3487.309814453125
1191 3712.346435546875
1192 3177.420654296875
1193 3380.92626953125
1194 3200.978515625
1195 3610.031982421875
1196 3415.246826171875
1197 3282.26025390625
1198 3623.58740234375
1199 3545.0478515625
1200 3018.553466796875
1201 3746.4638671875
1202 4011.933837890625
1203 3221.901611328125
1204 3077.124755859375
1205 3576.723388671875
1206 3085.7412109375
1207 3685.2294921875
1208 3363.36865234375
1209 3629.441162109375
1210 4246.86669921875
1211 3589.464111328125
1212 3743.261962890625
1213 3289.92333984375
1214 3206.980712890625
1215 3216.11328125
1216 3166.27197265625
1217 3497.52685546875
1218 3167.162109375
1219 3848.3955078125
1220 3565.094970703125
1221 3446.90869140625
1222 3350.671875
1223 3267.3623046875
1224 3238.716552734375
1225 3548.7861328125
1226 4029.30078125
1227 3733.15771484375
1228 3641.5

1559 2938.784423828125
1560 4087.743896484375
1561 3299.136962890625
1562 3050.56298828125
1563 3303.277099609375
1564 3319.4638671875
1565 3363.784912109375
1566 3378.6572265625
1567 3606.4736328125
1568 3381.5595703125
1569 3032.078369140625
1570 3344.0166015625
1571 3189.23486328125
1572 3204.23193359375
1573 3668.9814453125
1574 3168.69921875
1575 2899.67431640625
1576 3629.79638671875
1577 3086.732177734375
1578 3536.133056640625
1579 3532.90673828125
1580 3502.2421875
1581 3474.28076171875
1582 3731.249267578125
1583 3643.58544921875
1584 3522.6943359375
1585 3122.390869140625
1586 3438.365966796875
1587 3437.498046875
1588 3586.75
1589 3840.343505859375
1590 3620.899169921875
1591 3467.65380859375
1592 3014.553955078125
1593 3046.02880859375
1594 3165.509765625
1595 3400.181640625
1596 3265.002685546875
1597 3642.093017578125
1598 3451.924560546875
1599 3442.646728515625
1600 3567.16357421875
1601 3524.59814453125
1602 3482.265869140625
1603 2940.877685546875
1604 4117.110351562

1937 3491.050537109375
1938 3559.950927734375
1939 3897.9755859375
1940 3487.771240234375
1941 3523.1435546875
1942 3303.203857421875
1943 3659.370361328125
1944 3364.033203125
1945 3241.136962890625
1946 3362.692626953125
1947 4076.357666015625
1948 3553.791015625
1949 3481.0986328125
1950 3382.88037109375
1951 2979.09228515625
1952 3717.943359375
1953 3812.828125
1954 3648.545166015625
1955 3752.845458984375
1956 3324.34619140625
1957 3098.615478515625
1958 3476.834228515625
1959 4135.1533203125
1960 3333.6357421875
1961 3310.83544921875
1962 3089.3505859375
1963 3915.60498046875
1964 3966.357666015625
1965 2979.09423828125
1966 3844.112060546875
1967 3146.616943359375
1968 3324.31591796875
1969 3291.279296875
1970 3439.203857421875
1971 3619.741455078125
1972 3492.94970703125
1973 3316.8173828125
1974 3015.414306640625
1975 3152.0634765625
1976 2775.473876953125
1977 3606.402099609375
1978 3531.088623046875
1979 3465.5283203125
1980 3348.652099609375
1981 3220.78466796875
1982 3147.

2324 3072.868408203125
2325 3190.1865234375
2326 3706.293212890625
2327 3965.067626953125
2328 3470.34228515625
2329 3229.38037109375
2330 3678.412353515625
2331 4209.19921875
2332 3549.016357421875
2333 3541.258544921875
2334 3734.876220703125
2335 3271.068603515625
2336 3309.163330078125
2337 3524.404052734375
2338 4276.474609375
2339 3725.2890625
2340 3670.148193359375
2341 3632.44384765625
2342 3515.50341796875
2343 3263.074951171875
2344 4104.22412109375
2345 3654.03564453125
2346 3364.409912109375
2347 2978.88525390625
2348 3313.61962890625
2349 3560.6025390625
2350 3956.723388671875
2351 3224.92138671875
2352 3673.0126953125
2353 3362.2275390625
2354 3342.479736328125
2355 3217.14404296875
2356 3022.739013671875
2357 3707.41845703125
2358 3628.481689453125
2359 2930.49169921875
2360 3435.3564453125
2361 2958.267333984375
2362 3984.3818359375
2363 3404.54443359375
2364 3040.267333984375
2365 3547.273193359375
2366 4151.544921875
2367 3678.071044921875
2368 3635.123291015625
2369 

2709 3722.7158203125
2710 3519.963623046875
2711 3454.44873046875
2712 2943.6796875
2713 3201.431640625
2714 3501.112060546875
2715 3073.111083984375
2716 3623.9775390625
2717 3070.55126953125
2718 3889.047607421875
2719 3645.72314453125
2720 3696.984375
2721 3497.4228515625
2722 3524.294189453125
2723 3542.106689453125
2724 3333.676025390625
2725 3337.55126953125
2726 3595.75732421875
2727 2999.545166015625
2728 3477.162841796875
2729 3897.696533203125
2730 3504.697265625
2731 3655.301025390625
2732 3320.478515625
2733 3171.262451171875
2734 3606.31005859375
2735 3136.25927734375
2736 3066.07421875
2737 3611.98828125
2738 3457.1103515625
2739 3605.2275390625
2740 3125.9091796875
2741 3287.82958984375
2742 3630.56982421875
2743 3577.09521484375
2744 3093.647705078125
2745 3017.667236328125
2746 3825.134521484375
2747 3172.895263671875
2748 2970.291259765625
2749 3793.286376953125
2750 3144.131103515625
2751 3444.98095703125
2752 3363.966552734375
2753 3877.0986328125
2754 4198.96533203

In [5]:
df = pd.read_csv('test.csv') ## Import data
print(len(reqd_feat))
df = df[reqd_feat]
a = df.to_numpy()
a[np.isnan(a).any(axis=1)] = 0

with torch.no_grad():
    x = torch.tensor(a, dtype=torch.float32)
    y_pred = model(x)
    y_pred = y_pred.numpy()
    
print(y_pred)

15
[[2.5896608e+08]
 [2.7475344e+08]
 [2.9813462e+08]
 ...
 [3.2155258e+08]
 [2.4693320e+08]
 [2.8039712e+08]]
